# Library

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import Callback
from keras import layers
from keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from pandas import read_csv

import pandas as pd
import numpy as np
np.set_printoptions(linewidth=np.inf)

# Dataset

In [2]:
# LOAD dataset
dataset_path = './dataset_minecraft.csv'
df = read_csv(dataset_path)
df = df.dropna()
df = df.drop_duplicates()
print("\"{}\" is loaded succesfully".format(dataset_path))

dataset = df['Text Clean'].to_numpy()
label = df['Sentiment'].to_numpy()

"./dataset_minecraft.csv" is loaded succesfully


### Data Splitting

In [3]:
# feature_train, feature_test, label_train, label_test = train_test_split(dataset, label, test_size=0.3, random_state=42)

# Features Extraction

### Dataset

In [4]:
# MAX WORD LENGTH
word_len = []
for word in dataset:
    word_len.append(len(word.split()))
max_word_length = max(word_len)
print("Max Word Length :\033[1m", max_word_length)
####################################################################################

# UNIQUE WORDS
def counter_word(texts):
    count = Counter()
    for text in texts.values:
        for word in text.split():
            count[word] += 1
    return count

counter = counter_word(df['Text Clean'])
num_unique_words = len(counter) # 14000
print("\nUnique words :\033[1m", num_unique_words)
####################################################################################

# TOKENIZING
# tokenizing dataset
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(dataset)
# print(tokenizer.word_index.keys())
# print(tokenizer.word_index.values())
####################################################################################

# FEATURE ENCODER
# dictionary for word
word_index = tokenizer.word_index
# turn dataset to sequence
dataset_seq = tokenizer.texts_to_sequences(dataset)
print("\n===Feature Encoder Test=== ")
print("Text Original : ", dataset[6])
print("Text Encoded  : ", dataset_seq[6])
####################################################################################

# FEATURE DECODER
# Flip word dictionary (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])
decoded_text = decode(dataset_seq[6])
print("\n===Feature Decoder Test=== ")
print("Text Sequence : ", dataset_seq[6])
print("Text Decoded  : ", decoded_text)
####################################################################################

# Padding
dataset_padded = pad_sequences(dataset_seq, maxlen=max_word_length, padding="pre", truncating="pre")
print("\nPadding Test : ")
print("Text Ori   : ", dataset[6])
print("Text Token : ", dataset_seq[6])
print("Token Pad  : ", dataset_padded[6])
####################################################################################

Max Word Length : 88

Unique words : 14000

===Feature Encoder Test=== 
Text Original :  game nya bagus serutapi freeze kamera detik menggangu survivalsaya bikin lag frame drop freeze kamera baik cepat baik
Text Encoded  :  [1, 2, 3, 1548, 132, 466, 143, 346, 4902, 39, 42, 81, 78, 132, 466, 6, 133, 6]

===Feature Decoder Test=== 
Text Sequence :  [1, 2, 3, 1548, 132, 466, 143, 346, 4902, 39, 42, 81, 78, 132, 466, 6, 133, 6]
Text Decoded  :  game nya bagus serutapi freeze kamera detik menggangu survivalsaya bikin lag frame drop freeze kamera baik cepat baik

Padding Test : 
Text Ori   :  game nya bagus serutapi freeze kamera detik menggangu survivalsaya bikin lag frame drop freeze kamera baik cepat baik
Text Token :  [1, 2, 3, 1548, 132, 466, 143, 346, 4902, 39, 42, 81, 78, 132, 466, 6, 133, 6]
Token Pad  :  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

### Label

In [5]:
# LABEL ENCODE
encoder = OneHotEncoder(sparse_output=False)
label_one_hot = encoder.fit_transform(label.reshape(-1, 1))
print("\n===Label Encoder Test=== ")
print(encoder.get_feature_names_out())
print("Onehot \"{}\" : {}".format(df['Sentiment'][0], label_one_hot[0]))


===Label Encoder Test=== 
['x0_negative' 'x0_neutral' 'x0_positive']
Onehot "negative" : [1. 0. 0.]


### Oversampling

In [6]:
smote = SMOTE()
x_over, y_over = smote.fit_resample(dataset_padded, label_one_hot)

In [7]:
# BEFORE OVERSAMPLING
df.Sentiment.value_counts()

Sentiment
positive    7034
negative    6122
neutral     1481
Name: count, dtype: int64

In [8]:
# AFTER OVERSAMPLING
def decoderOnehot(data):
    label = encoder.inverse_transform([data])
    return label[0][0]
    
new_df = pd.DataFrame(list(zip(x_over, y_over)), columns=['features', 'label'])
new_df['label'] = new_df['label'].apply(decoderOnehot)
new_df['label'].value_counts()
# new_df

label
negative    7034
positive    7034
neutral     7034
Name: count, dtype: int64

### Splitting

In [9]:
print(x_over.shape, y_over.shape)
X_train, X_test, y_train, y_test = train_test_split(x_over, y_over, test_size=0.3, random_state=42)

(21102, 88) (21102, 3)


In [10]:
train_df = pd.DataFrame(zip(y_train), columns=['label'])
train_df['label'] = train_df['label'].apply(decoderOnehot)
train_df.value_counts()

label   
neutral     4945
negative    4916
positive    4910
Name: count, dtype: int64

In [11]:
test_df = pd.DataFrame(zip(y_test), columns=['label'])
test_df['label'] = test_df['label'].apply(decoderOnehot)
test_df.value_counts()

label   
positive    2124
negative    2118
neutral     2089
Name: count, dtype: int64

# Model

In [12]:
# DEFINE CALLBACK
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.92 and logs.get('val_accuracy') >= 0.92):
            self.ltm.stop_training=True
            print('\nModel telah mencapai akurasi 92%')

In [22]:
# Feature Extraction
wordEmbedding = layers.Embedding(
                        input_dim=num_unique_words, 
                        output_dim=64, 
                        input_length=max_word_length,
                        mask_zero=True                  # cuz i use zero padding so model will ignore 0 value
                        )

lstm = Sequential([
    wordEmbedding,
    # layers.Bidirectional(layers.LSTM(88, dropout=0.2, kernel_regularizer=l2(0.01))),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.3, kernel_regularizer=l2(0.01))),
    layers.Bidirectional(layers.LSTM(64, dropout=0.3, kernel_regularizer=l2(0.01))),
    # layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])
lstm.summary()

lstm.compile(loss=categorical_crossentropy,
             optimizer=Adam(learning_rate=0.00001),
             metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 88, 64)            896000    
                                                                 
 bidirectional_13 (Bidirect  (None, 88, 128)           66048     
 ional)                                                          
                                                                 
 bidirectional_14 (Bidirect  (None, 128)               98816     
 ional)                                                          
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 3)                

In [24]:
history_lstm = lstm.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        batch_size=64, 
                        epochs=500, 
                        callbacks=myCallback())

Epoch 1/500
231/231 [==============================] - 104s 451ms/step - loss: 6.2515 - accuracy: 0.3262 - val_loss: 6.1132 - val_accuracy: 0.3262
Epoch 2/500
231/231 [==============================] - 83s 360ms/step - loss: 5.9807 - accuracy: 0.3411 - val_loss: 5.8489 - val_accuracy: 0.3511
Epoch 3/500
231/231 [==============================] - 81s 351ms/step - loss: 5.7224 - accuracy: 0.3486 - val_loss: 5.5966 - val_accuracy: 0.3772
Epoch 4/500
231/231 [==============================] - 81s 353ms/step - loss: 5.4759 - accuracy: 0.3537 - val_loss: 5.3555 - val_accuracy: 0.3876
Epoch 5/500
231/231 [==============================] - 80s 345ms/step - loss: 5.2397 - accuracy: 0.3604 - val_loss: 5.1242 - val_accuracy: 0.3781
Epoch 6/500
231/231 [==============================] - 79s 341ms/step - loss: 5.0131 - accuracy: 0.3665 - val_loss: 4.9016 - val_accuracy: 0.3846
Epoch 7/500
231/231 [==============================] - 79s 341ms/step - loss: 4.7953 - accuracy: 0.3807 - val_loss: 4.6889 

KeyboardInterrupt: 